In [1]:
#import needed libraries
from sqlalchemy import create_engine, text
import pandas as pd
import os
import time

In [2]:
#Destination config
username = ''
password = ''
hostname = ''
port = ''
database_destination = ''
database_source = ''
prefix = ''
list_table_name = []
file_dir_table = r'C:\Users\Long\Untitled Folder\table.txt'
file_dir_config = r'C:\Users\Long\Untitled Folder\config.txt'
file_dir_log = r'C:\Users\Long\Untitled Folder'+f'\cdc_log_{database_source}.txt'

In [3]:
def take_table_name_from_file(file_dir):
    with open(file_dir) as f:
        lines = f.readlines()
        f.close()
    list_table_tmp = lines[0].split(",")
    for item in list_table_tmp:
        if(item[0] == ' '):
            item = item[1:]
        list_table_name.append(item)
    f.close()

In [4]:
def take_config_from_file(file_dir):
    with open(file_dir) as f:
        lines = f.readlines()
        f.close()
    return lines[1]

In [5]:
take_table_name_from_file(file_dir_table)

In [6]:
#Use file config to take config
username, password, hostname, port, database_destination, database_source, prefix \
= take_config_from_file(file_dir_config).split('\t')

In [7]:
engine_source = create_engine(f'mssql+pymssql://{username}:{password}@{hostname}:{port}/{database_source}') 
engine_target = create_engine(f'mssql+pymssql://{username}:{password}@{hostname}:{port}/{database_destination}') 


In [8]:
for table_name in list_table_name:
    sql = "TRUNCATE TABLE " + f"{table_name}"
    df = pd.read_sql(f'Select * from {table_name}', engine_source)
    for index, row in df.iterrows():
        with engine_target.begin() as conn:
            conn.execute(text(sql))
        mod = pd.DataFrame(row.to_frame().T)
        table_name = table_name[4:]
        mod.to_sql(f"{table_name}", engine_target, if_exists='append', index=False)
        str_log = "Row Insert "
        for col in mod.columns:
            str_log  = str_log + mod[col].astype(str) + " "
        f = open(f"ETL_{table_name}_logs", "a")
        f.write(str_log.to_string() + "\n")
        f.close()